In [1]:
import cv2
import numpy as np
import imutils
from collections import deque
import time
from imutils.video import VideoStream

In [2]:
greenLower = (29,86,6)
greenUpper = (64,255,255)
pts = deque(maxlen=100)
cap = cv2.VideoCapture(0)
time.sleep(2.0)

while(True):
    ret,frame = cap.read()
    if(frame is None):
        break
    frame = imutils.resize(frame,width=600)
    blurred = cv2.GaussianBlur(frame,(11,11),0)
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv,greenLower,greenUpper)
    mask = cv2.erode(mask,None,iterations=2)
    mask = cv2.dilate(mask,None,iterations=2)
    
    cnts = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    if(len(cnts) > 0):
        c = max(cnts, key=cv2.contourArea)
        ((x,y),radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M['m10']/M['m00']),int(M['m01']/M['m00']))
        if(radius > 10):
            cv2.circle(frame,(int(x),int(y)),int(radius),(0,255,255),2)
            cv2.circle(frame,center,5,(0,0,255),-1)
    pts.appendleft(center)
    
    for i in range(1,len(pts)):
        if(pts[i-1] is None or pts[i] is None):
            continue
        thickness = int(np.sqrt(100/float(i+1))*2.5)
        cv2.line(frame,pts[i-1],pts[i],(0,0,255),thickness)
    frame =  cv2.flip(frame,1)
    cv2.imshow("frame",frame)
    if(cv2.waitKey(1) == 13):
        cap.release()
        break
cv2.destroyAllWindows()